# Project: Wrangling and Analyze Data

In [13]:
import pandas as pd
import requests
import tweepy
import json

## Data Gathering
In the cell below, gather **all** three pieces of data for this project and load them in the notebook. **Note:** the methods required to gather each data are different.
1. Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)

In [2]:
df1 = pd.read_csv('twitter-archive-enhanced.csv')

2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

In [5]:
image = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv', allow_redirects=True)
open('image_predictions.tsv', 'wb').write(image.content)

df2 = pd.read_csv('image_predictions.tsv', sep = '\t')
df2.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

**How to set-up Twitter API**

Temporary credentials:

* Request Token === oauth_token
* Request Token Secret === oauth_token_secret
* oauth_verifier


Access token

1530238502633459713-O9uxuQDSxmDfk3IN0VwA5xvEt8hogg

Access token Secret 

ekH8zzk0dVgkVbV6LzZ25DqaFmSKjYJ5LajtrHbOoM65i

Bearer Token

AAAAAAAAAAAAAAAAAAAAAKfegQEAAAAAjm7HcbIXSWH9dVxFMaVcFkoZmN0%3DLm2PybknA4QiOzh4cizDfKJvS7e6hqYFzYqxZzbteLmhiVp7jJ

API Key

1BE18QbbWnCvkpCpasMtixY1T

API Key Secret

KJ9boyoEVOWmNNeVTEpNpvuQkVPKFOjl9ymuXtyzDhrNp8zGJL



In [10]:
api_key = consumer_key = '1BE18QbbWnCvkpCpasMtixY1T'
api_secret = consumer_secret = 'KJ9boyoEVOWmNNeVTEpNpvuQkVPKFOjl9ymuXtyzDhrNp8zGJL'
access_token = '1530238502633459713-O9uxuQDSxmDfk3IN0VwA5xvEt8hogg'
access_token_secret = 'ekH8zzk0dVgkVbV6LzZ25DqaFmSKjYJ5LajtrHbOoM65i'

auth = tweepy.OAuth1UserHandler(
   consumer_key, consumer_secret, access_token, access_token_secret
)

api = tweepy.API(auth, wait_on_rate_limit = True)

**Matching DF1 with Twitter API**

Now we must download new info based on the ids of our previous df1 database.

In [11]:
#Matched tweets
tweets = []
# Mistmatched tweets
n_tweets = []

for tweet_id in df1['tweet_id']:   
    try:
        tweets.append(api.get_status(tweet_id))
    except Exception as e:
        n_tweets.append(tweet_id)
 
print("The list of tweets" ,len(tweets))
print("The list of tweets no found" , len(n_tweets))

Rate limit reached. Sleeping for: 390
Rate limit reached. Sleeping for: 393


The list of tweets 2327
The list of tweets no found 29


**JSON Treatment**

This part is basically uniting all json parts from each tweet and uniting them into a single json

In [83]:
dic = []

#getting the json
for json_tweet in tweets:
    dic.append(json_tweet)

#writing json to a txt
with open('json_tweet.txt', 'w') as file:
        file.write(json.dumps([dic[0]._json for status in dic], indent=4))

In [104]:

#identifying and labeling information from txt
list = []

with open('json_tweet.txt', encoding='utf-8') as json_file:  
    alldics = json.load(json_file)
    
    for dic in alldics:
        id = dic['id']
        text = dic['text']
        url = text[text.find('https'):]
        favorites = dic['favorite_count']
        retweets = dic['retweet_count']
        followers = dic['user']['followers_count']
        friends = dic['user']['friends_count']
        whole_source = dic['source']
        only_device = whole_source[whole_source.find('rel="nofollow">') + 15:-4]
        source = only_device
        retweeted_status = dic['retweeted_status'] = dic.get('retweeted_status', 'Original tweet')

        if retweeted_status == 'Original tweet':
            url = url
        else:
            retweeted_status = 'This is a retweet'
            url = 'This is a retweet'

        list.append({'tweet_id': str(id),
                             'favorite_count': int(favorites),
                             'retweet_count': int(retweets),
                             'followers_count': int(followers),
                             'friends_count': int(friends),
                             'url': url,
                             'source': source,
                             'retweeted_status': retweeted_status,
                            })
                            
        df3 = pd.DataFrame(list, columns = ['tweet_id', 'favorite_count','retweet_count', 
                                                           'followers_count', 'friends_count','source', 
                                                           'retweeted_status', 'url'])

df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2327 entries, 0 to 2326
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tweet_id          2327 non-null   object
 1   favorite_count    2327 non-null   int64 
 2   retweet_count     2327 non-null   int64 
 3   followers_count   2327 non-null   int64 
 4   friends_count     2327 non-null   int64 
 5   source            2327 non-null   object
 6   retweeted_status  2327 non-null   object
 7   url               2327 non-null   object
dtypes: int64(4), object(4)
memory usage: 145.6+ KB


## Assessing Data
In this section, detect and document at least **eight (8) quality issues and two (2) tidiness issue**. You must use **both** visual assessment
programmatic assessement to assess the data.

**Note:** pay attention to the following key points when you access the data.

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This [unique rating system](http://knowyourmeme.com/memes/theyre-good-dogs-brent) is a big part of the popularity of WeRateDogs.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.



### Visual Assessment

In [107]:
df1.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [108]:
df2.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [109]:
df3.head()

,tweet_id,favorite_count,retweet_count,followers_count,friends_count,source,retweeted_status,url
0,892420643555336193,33713,6975,9352206,21,Twitter for iPhone,Original tweet,https://t.co/MgUWQ76dJU
1,892420643555336193,33713,6975,9352206,21,Twitter for iPhone,Original tweet,https://t.co/MgUWQ76dJU
2,892420643555336193,33713,6975,9352206,21,Twitter for iPhone,Original tweet,https://t.co/MgUWQ76dJU
3,892420643555336193,33713,6975,9352206,21,Twitter for iPhone,Original tweet,https://t.co/MgUWQ76dJU
4,892420643555336193,33713,6975,9352206,21,Twitter for iPhone,Original tweet,https://t.co/MgUWQ76dJU


### Programmatic Assessment

In [110]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [111]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [112]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2327 entries, 0 to 2326
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tweet_id          2327 non-null   object
 1   favorite_count    2327 non-null   int64 
 2   retweet_count     2327 non-null   int64 
 3   followers_count   2327 non-null   int64 
 4   friends_count     2327 non-null   int64 
 5   source            2327 non-null   object
 6   retweeted_status  2327 non-null   object
 7   url               2327 non-null   object
dtypes: int64(4), object(4)
memory usage: 145.6+ KB


### Quality issues
1.

2.

3.

4.

5.

6.

7.

8.

### Tidiness issues
1.

2.

## Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

In [ ]:
# Make copies of original pieces of data


### Issue #1:

#### Define:

#### Code

#### Test

### Issue #2:

#### Define

#### Code

#### Test

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

### Insights:
1.

2.

3.

### Visualization